# Default Import

In [1]:
from ris import pysqldb
from IPython.display import Markdown, clear_output
from sqlalchemy import create_engine
import ris
import getpass
import datetime 
import pandas as pd
import numpy as np
import os
import requests

timestamp = datetime.datetime.now().strftime('%Y-%m-%d %H:%M')
ts = datetime.datetime.now().strftime('%Y-%m-%d')

Markdown('<strong>Notebook run on: {} | by {} | Using ris library version: {} | File Location: {}'.format(
    timestamp, getpass.getuser(), ris.__version__, os.getcwd()
))


<strong>Notebook run on: 2020-07-28 15:11 | by soge | Using ris library version: 1.5.6 | File Location: E:\RIS\Staff Folders\Samuel\Requests\AG\SIP treatments request

# Db connections

In [2]:
try: 
    db.params['user']
except:
    db = pysqldb.DbConnect(type='PG', server='dotpgsql01', database='sip')


Additional database connection details required:
User name (sip):soge
Password (sip)········
Database connection (PG) to sip on dotpgsql01 - user: soge 
Connection established 2020-07-28 15:11:32


In [3]:
#sip = pysqldb.DbConnect(server='dotpgsql01', database='sip', type='PG', user = db.params['user'], password = db.params['password'])
#forms = pysqldb.DbConnect(server='dot55sql01', database='forms', type='MS', user = 'arcgis', password = 'arcgis')

# Pedestrian Treatments Desired:

### New Crossings
### Sidewalk expansions
### Ped Plaza
### Neckdowns (all types)
### Refuge Islands
### Other Ped Space


In [7]:
sip_treatments = db.dfquery(""" -- Querying questions table to get qid's for specific treatments
                                -- Querying questions path table to get all descendants of desired treatments

                                WITH data as (
                                SELECT sq.qid, sq.question, sqp.*
                                FROM sip_questions sq
                                JOIN sip_questions_path sqp
                                ON sq.qid = sqp.ancestor
                                WHERE sq.question in ('New Crossings', 'Sidewalk Expansions', 'Ped Plaza', 'Neckdown', 'Ped Refuge Islands', 'Other Ped Space')
                                )

                                -- Needed: pid, tid, pjct_name, sip_year, start_date, end_date, pm, unit
                                -- Also grabbing segmentids and nodeids
                                -- Joining to lookup table (sl) to get status description
                                -- Joining to lookup table (su) to get unit description

                                ,data2 as(

                                SELECT tid, question treatment, pid_fk, pjct_name, sip_year, start_date, end_date, pm, unit, su.description, sl.description status_desc, segmentid::int, nodeid::int 
                                FROM( SELECT  distinct st.tid, data.question, st.pid_fk, sp.pjct_name, sp.sip_year, sp.start_date, sp.end_date, sp.pm, sp.unit, sp.status::int, segmentid::int, nodeid::int
                                      FROM sip_treatments st
                                      JOIN sip_projects sp
                                      ON st.pid_fk=sp.pid
                                      JOIN public.sip_treatments_geo stg
                                      ON st.tid=stg.tid_fk
                                      JOIN data 
                                      ON st.breadcrumbid = data.descendant
                                      and sp.status::int = 15
                                ) sips
                                JOIN sip_lookup sl
                                ON sips.status = sl.lookupid
                                JOIN sip_lookup su
                                ON sips.unit = su.lookupid::varchar
                                )

                                SELECT * 
                                FROM data2
                                WHERE nodeid = 0

                                UNION

                                -- Grabbing all segments from clion ending in treatment nodeids

                                SELECT * FROM (
                                SELECT tid, treatment, pid_fk, pjct_name, sip_year, start_date, end_date, pm, unit, description, status_desc, cl.segmentid::int, nodeid 
                                FROM data2 d2
                                JOIN clion cl
                                ON d2.nodeid = cl.nodeidto::int
                                WHERE d2.nodeid != 0

                                UNION

                                SELECT tid, treatment, pid_fk, pjct_name, sip_year, start_date, end_date, pm, unit, description, status_desc, cl.segmentid::int, nodeid
                                FROM data2 d2
                                JOIN clion cl
                                ON d2.nodeid = cl.nodeidfrom::int
                                WHERE d2.nodeid != 0
                                )nodes_to_segments
                                ORDER BY segmentid

                                """)

In [8]:
sip_treatments.to_csv('sip_pedestrian_treatments.csv', index = False)

#### pysqldb is not working so shapefile created via qgis